<img src="tec-logo.jpg" width=200> 

<img src="EP_fig0.svg" width=500> 


## In this problem we are interested in  determining the optimal investment capacity in various types of power plants so as to meet  period demands for electricity. Four power plants (P1,P2,P3,P4) are available and, in a typical  year,  they can operate in three different demand operating modes (M1,M2,M3). In each one of the 3 operating modes there is an energy demand (D1,D2,D3) to be met. There is a budget constraint and also a constraint on the minimum total power capacity. Solve the optimal power capacity problem taking into account that power demands at Modes 2 and 3 remain constant (3 and 2 GW, respectively) along the year. However, power demand at Mode 1 is uncertain. Therefore, 3 power uncertainty scenarios (S1,S2,S3) will be considered for addressing power variability in Mode 1. Power demand at these uncertain scenarios is given  as follows: 3, 5 and 7 GW, respectively.

## Determine: The total installed power amount in each plant, as well as the amount of power output at each plant for each operating model such all operating constraints are met and minimizing the total anual cost.


<img src="EP_fig1.svg" width=700> 

| Plant 	| Investment cost 	|
|:-----:	|:---------------:	|
|   1   	|        10       	|
|   2   	|        7        	|
|   3   	|        16       	|
|   4   	|        6        	|

### Operating cost

| Plant 	| Mode 1 	| Mode 2 	| Mode 3 	|
|:-----:	|:------:	|:------:	|:------:	|
|   1   	|   40   	|   24   	|    4   	|
|   2   	|   45   	|   27   	|   4.5  	|
|   3   	|   32   	|  19.2  	|   3.2  	|
|   4   	|   55   	|   33   	|   5.5  	|

### Problem statement was taken from: Louveaux, F V, and Smeers, Y, Optimal Investments for Electricity Generation: A Stochastic Model and a Test Problem. In Ermoliev, Y, and Wets, R J, Eds, Numerical Techniques for Stochastic Optimization Problems. Springer Verlag, 1988, pp. 445-452.


<img src="EP_fig2.svg" width=700> 

In [1]:
using JuMP, Gurobi, Plots, Printf

In [2]:
energy_planning_model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only


In [3]:
np   = 4                              # Number of power plants
ns   = 3                              # Number of operating modes
nu   = 3                              # Number of uncertain energy demands in scenario 1 
m    = 12                             # Minimum installed capacity
b    = 120                            # Budget limit
c    = zeros(np)                      # Investment cost
d    = zeros(ns)                      # Energy demand
dvar = zeros(nu)                      # Variable energy demands in scenario 1
f    = Array{Float64}(undef, np, ns)  # Operating cost
npp  = 1:np
nss  = 1:ns
nuu  = 1:nu

c    = [10 7 16 6]
d    = [0 3 2]
dvar = [3 5 7]
f    = [40  24    4   ;
        45  27    4.5 ;
        32  19.2  3.2 ;
        55  33    5.5 ];

### First, let us solve the fully Deterministic energy planning problem

<img src="EP_fig3.svg" width=800> 

In [4]:
@variable(energy_planning_model, x[i in npp] >=  0)               # Installed capacity
@variable(energy_planning_model, y[in in npp, j in nss] >= 0 )   # Operating level
@variable(energy_planning_model, cost >=0) ;

In [5]:
@constraint(energy_planning_model, mincap, sum(x[i] for i in npp) >= m ) ;

In [6]:
@constraint(energy_planning_model, bbal, sum(c[i]*x[i] for i in npp) <= b ) ;

In [7]:
@constraint(energy_planning_model, powbal[i in npp], sum(y[i,j] for j in nss) <= x[i] ) ;

In [8]:
@constraint(energy_planning_model, dembal[j in nss], sum(y[i,j] for i in npp) >= d[j] ) ;

In [9]:
@constraint(energy_planning_model,defcost,cost==sum(c[i]*x[i] for i in npp)+sum(f[i,j]*y[i,j] for i in npp, j in nss));

In [10]:
@objective(energy_planning_model, Min, cost) ;

In [11]:
set_normalized_rhs(dembal[1], dvar[1])
JuMP.optimize!(energy_planning_model)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 10 rows, 17 columns and 53 nonzeros
Model fingerprint: 0x254ef62b
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 9 rows, 16 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.300000e+01   0.000000e+00      0s
       9    2.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.00 seconds
Optimal objective  2.930000000e+02


In [12]:
@printf(" Cost : %f\n", JuMP.objective_value(energy_planning_model))
println("---------------------------------")
println("  Plant   Installed Capacity")
println("---------------------------------")
for i = 1:np
    @printf("    %i          %f\n", i, JuMP.value(x[i]))
end
println("---------------------------------")

println("\n")
println("---------------------------------------------")
println("Plant   Mode    Operating Level")
println("---------------------------------------------")
for i = 1:np
    for j = 1:ns
            if JuMP.value(y[i,j]) > 0  
                @printf("  %i       %i     %8.2f\n", i,j, JuMP.value(y[i,j]))
            end
    end
end
println("--------------------------------------------")

 Cost : 293.000000
---------------------------------
  Plant   Installed Capacity
---------------------------------
    1          3.000000
    2          0.000000
    3          3.000000
    4          6.000000
---------------------------------


---------------------------------------------
Plant   Mode    Operating Level
---------------------------------------------
  1       2         3.00
  3       1         3.00
  4       3         2.00
--------------------------------------------


In [13]:
set_normalized_rhs(dembal[1], dvar[2])
JuMP.optimize!(energy_planning_model)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 10 rows, 17 columns and 53 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7700000e+02   8.750000e-01   0.000000e+00      0s
       3    3.7866667e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds
Optimal objective  3.786666667e+02


In [14]:
@printf(" Cost : %f\n", JuMP.objective_value(energy_planning_model))
println("---------------------------------")
println("  Plant   Installed Capacity")
println("---------------------------------")
for i = 1:np
    @printf("    %i          %f\n", i, JuMP.value(x[i]))
end
println("---------------------------------")

println("\n")
println("---------------------------------------------")
println("Plant   Mode    Operating Level")
println("---------------------------------------------")
for i = 1:np
    for j = 1:ns
            if JuMP.value(y[i,j]) > 0  
                @printf("  %i       %i     %8.2f\n", i,j, JuMP.value(y[i,j]))
            end
    end
end
println("--------------------------------------------")

 Cost : 378.666667
---------------------------------
  Plant   Installed Capacity
---------------------------------
    1          0.833333
    2          3.000000
    3          4.166667
    4          4.000000
---------------------------------


---------------------------------------------
Plant   Mode    Operating Level
---------------------------------------------
  1       1         0.83
  2       2         3.00
  3       1         4.17
  4       3         2.00
--------------------------------------------


In [15]:
set_normalized_rhs(dembal[1], dvar[3])
JuMP.optimize!(energy_planning_model)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 10 rows, 17 columns and 53 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6933333e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  4.693333333e+02


In [16]:
@printf(" Cost : %f\n", JuMP.objective_value(energy_planning_model))
println("---------------------------------")
println("  Plant   Installed Capacity")
println("---------------------------------")
for i = 1:np
    @printf("    %i          %f\n", i, JuMP.value(x[i]))
end
println("---------------------------------")

println("\n")
println("---------------------------------------------")
println("Plant   Mode    Operating Level")
println("---------------------------------------------")
for i = 1:np
    for j = 1:ns
            if JuMP.value(y[i,j]) > 0  
                @printf("  %i       %i     %8.2f\n", i,j, JuMP.value(y[i,j]))
            end
    end
end
println("--------------------------------------------")

 Cost : 469.333333
---------------------------------
  Plant   Installed Capacity
---------------------------------
    1          4.166667
    2          3.000000
    3          2.833333
    4          2.000000
---------------------------------


---------------------------------------------
Plant   Mode    Operating Level
---------------------------------------------
  1       1         4.17
  2       2         3.00
  3       1         2.83
  4       3         2.00
--------------------------------------------


### Now, let us solve the Stochastic energy planning problem

### Uncertainty Source: Energy Demand in Mode 1 of  3 operating modes

<img src="EP_fig5.svg" width=800> 

| Mode 	| Probabilty 	| Demand 	|
|:----:	|:----------:	|:------:	|
|   1  	|     0.3    	|    3   	|
|   2  	|     0.4    	|    5   	|
|   3  	|     0.3    	|    7   	|


<img src="EP_fig4.svg" width=800> 

In [17]:
energy_planning_model_s = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only


In [18]:
ds    = Array{Float64}(undef, ns, nu)  # Stochastic demand
alpha = zeros(ns)                      # Scenario probability

alpha = [0.3  0.4  0.3]   # Uncertainty scenario probability
ds    = [3  5  7  ;       # Demand Mode 1 (Uncertainty source)
         3  3  3  ;       # Demand Mode 2
         2  2  2  ];      # Demand Mode 3

In [19]:
@variable(energy_planning_model_s, xs[i in npp] >=  0)                          # Installed capacity
@variable(energy_planning_model_s, ys[in in npp, j in nss, s in nuu] >= 0 ) ;   # Uncertain operating level
@variable(energy_planning_model_s, costs >=0) ;

In [20]:
@constraint(energy_planning_model_s, mincaps, sum(xs[i] for i in npp) >= m ) ;

In [21]:
@constraint(energy_planning_model_s, bbals, sum(c[i]*xs[i] for i in npp) <= b ) ;

In [22]:
@constraint(energy_planning_model_s, powbals[i in npp, s in nuu], sum(ys[i,j,s] for j in nss) <= xs[i] ) ;

In [23]:
@constraint(energy_planning_model_s, dembals[j in nss, s in nuu], sum(ys[i,j,s] for i in npp) >= ds[j,s] ) ;

In [24]:
@constraint(energy_planning_model_s, defcosts, costs == sum(c[i]*xs[i] for i in npp) + sum(alpha[s]*f[i,j]*ys[i,j,s] 
                                                        for i in npp, j in nss, s in nuu)) ; 

In [25]:
@objective(energy_planning_model_s, Min, costs); 

In [26]:
JuMP.optimize!(energy_planning_model_s)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 24 rows, 41 columns and 133 nonzeros
Model fingerprint: 0x8ce1a205
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 23 rows, 40 columns, 92 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.575000e+01   0.000000e+00      0s
      25    3.8185333e+02   0.000000e+00   0.000000e+00      0s

Solved in 25 iterations and 0.00 seconds
Optimal objective  3.818533333e+02


In [27]:
@printf(" Stochastic Cost : %f\n", JuMP.objective_value(energy_planning_model_s))
println("---------------------------------")
println("  Plant   Installed Capacity")
println("---------------------------------")
for i = 1:np
    @printf("    %i          %f\n", i, JuMP.value(xs[i]))
end
println("---------------------------------")

println("\n")
println("---------------------------------------------")
println("Plant   Mode    Scenario    Operating Level")
println("---------------------------------------------")
for i = 1:np
    for j = 1:ns
        for s = 1:nu
            if JuMP.value(ys[i,j,s]) > 0  
                @printf("  %i       %i        %i       %8.2f\n", i,j,s, JuMP.value(ys[i,j,s]))
            else
            end
        end
    end
end
println("--------------------------------------------")

 Stochastic Cost : 381.853333
---------------------------------
  Plant   Installed Capacity
---------------------------------
    1          2.666667
    2          4.000000
    3          3.333333
    4          2.000000
---------------------------------


---------------------------------------------
Plant   Mode    Scenario    Operating Level
---------------------------------------------
  1       1        2           1.67
  1       1        3           2.67
  1       2        1           2.67
  1       2        2           1.00
  2       1        3           1.00
  2       2        2           2.00
  2       2        3           3.00
  2       3        1           2.00
  2       3        2           2.00
  3       1        1           3.00
  3       1        2           3.33
  3       1        3           3.33
  3       2        1           0.33
  4       3        3           2.00
--------------------------------------------
